In [27]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

In [29]:
# 1. 열 이름 정의
COLUMN_NAMES = ['area', 'perimeter', 'compactness', 'length', 'width', 'asymmetry', 'groove', 'target']

# 2. 파싱 함수 정
def parse_line(line):
    parts = line.strip().split()
    try:
        return [float(x) for x in parts]
    except ValueError:
        return None  # 변환 실패 시 None 반환

# 3. 유효한 줄(열이 8개인 줄)만 필터링하는 함수
def is_valid_row(row):
    return row is not None and len(row) == 8

# 4. 데이터셋 로딩 및 전처리
with open("C:/Users/jinwo/Downloads/seeds_dataset.txt", "r") as file:
    lines = file.readlines()

parsed_data = list(map(parse_line, lines))
filtered_data = list(filter(is_valid_row, parsed_data))

# 리스트를 TensorFlow Dataset으로 변환
dataset = tf.data.Dataset.from_tensor_slices(filtered_data)

# 5. TensorFlow Dataset -> Python 리스트로 변환
data_list = list(dataset.as_numpy_iterator())

# 6. pandas DataFrame으로 변환
df = pd.DataFrame(data_list, columns=COLUMN_NAMES)

df

,area,perimeter,compactness,length,width,asymmetry,groove,target
0,15.260000,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.880000,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.290000,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.840000,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.139999,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
...,...,...,...,...,...,...,...,...
205,12.190000,13.20,0.8783,5.137,2.981,3.631,4.870,3.0
206,11.230000,12.88,0.8511,5.140,2.795,4.325,5.003,3.0
207,13.200000,13.66,0.8883,5.236,3.232,8.315,5.056,3.0
208,11.840000,13.21,0.8521,5.175,2.836,3.598,5.044,3.0


In [31]:
# 타겟 설정 및 분리
X = df.drop('target', axis=1)
y = df['target'].astype(int) - 1  # 0부터 시작하도록 조정

# 클래스 분포 출력 (선택적 디버깅용)
print(y.value_counts())

# 원-핫 인코딩
y = to_categorical(y, num_classes=3)

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.2, random_state=42
)

# 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

target
0    70
1    70
2    70
Name: count, dtype: int64


In [33]:
# ========== 1. Sequential API ==========
model1 = Sequential()
model1.add(Dense(16, input_shape=(7,), activation='relu'))
model1.add(Dense(8, activation='relu'))
model1.add(Dense(3, activation='softmax'))

# 모델 요약
model1.summary()

C:\Users\jinwo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 16)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 291 (1.14 KB)

 Trainable params: 291 (1.14 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
print("Training Sequential Model...")
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Training Sequential Model...
Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step - accuracy: 0.8808 - loss: 0.2647 - val_accuracy: 0.9048 - val_loss: 0.3099
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9133 - loss: 0.2111 - val_accuracy: 0.9048 - val_loss: 0.3101
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9075 - loss: 0.2313 - val_accuracy: 0.9048 - val_loss: 0.3064
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8867 - loss: 0.2617 - val_accuracy: 0.9048 - val_loss: 0.3037
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9083 - loss: 0.2285 - val_accuracy: 0.9048 - val_loss: 0.3017
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9010 - loss: 0.2295 - val_accuracy: 0.9048 - val_loss: 0.2998
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9177 - loss: 0.2104 - val_accuracy: 0.9048 - val_loss: 0.3017
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9016 - loss: 0.2148 - val_accura

In [36]:
# ========== 2. Functional API ==========
inputs = Input(shape=(7,))
x = Dense(16, activation='relu')(inputs)
x = Dense(8, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

model2 = Model(inputs=inputs, outputs=outputs)

# 모델 요약
model2.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 7)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 16)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 291 (1.14 KB)

 Trainable params: 291 (1.14 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
print("\\nTraining Functional API Model...")
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

\nTraining Functional API Model...
Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.9047 - loss: 0.2090 - val_accuracy: 0.8810 - val_loss: 0.2858
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9225 - loss: 0.2096 - val_accuracy: 0.8810 - val_loss: 0.2758
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9268 - loss: 0.2028 - val_accuracy: 0.8810 - val_loss: 0.2698
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9364 - loss: 0.1920 - val_accuracy: 0.8810 - val_loss: 0.2683
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9328 - loss: 0.1970 - val_accuracy: 0.8810 - val_loss: 0.2637
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9400 - loss: 0.1685 - val_accuracy: 0.8810 - val_loss: 0.2620
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9402 - loss: 0.1761 - val_accuracy: 0.9048 - val_loss: 0.2572
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9240 - loss: 0.1949 - val_

In [44]:
# ========== 3. Subclassing API ==========
class SimpleMLP(Model):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dense2 = Dense(8, activation='relu')
        self.out = Dense(3, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.out(x)

model3 = SimpleMLP()

In [49]:
print("\\nTraining Subclassing Model...")
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

\nTraining Subclassing Model...
Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.4628 - loss: 1.1093 - val_accuracy: 0.5952 - val_loss: 1.0611
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5160 - loss: 1.0418 - val_accuracy: 0.5952 - val_loss: 1.0152
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5498 - loss: 1.0029 - val_accuracy: 0.5952 - val_loss: 0.9719
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5801 - loss: 0.9695 - val_accuracy: 0.5952 - val_loss: 0.9332
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5321 - loss: 0.9502 - val_accuracy: 0.6190 - val_loss: 0.8984
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6250 - loss: 0.8772 - val_accuracy: 0.6190 - val_loss: 0.8667
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6469 - loss: 0.8668 - val_accuracy: 0.6667 - val_loss: 0.8356
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7120 - loss: 0.8314 - val_acc